In [218]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Dataset upload**

In [219]:
# uploading while finding and imputing NaN values
#missing_values = ['NaN','--','na', 'N/A', 'ND']
df = pd.read_csv('../input/olympic-data/49.csv')#, na_values = missing_values)
#df = df.iloc[: , 1:] # dropping the first two colums (useless information)
df

**Introductory question 1 - Data cleaning**

In [220]:
# reviewing null values
df.isnull().sum()

In [221]:
# replacing the NaN values with mean (all numerical data)
index_names = ['Age', 'Height', 'Weight']
# index_names
for index in index_names :
    df[index].fillna(df[index].mean(), inplace = True)

In [222]:
# confirming null value elimination
df.isnull().sum()

**Introductory question 3 - BMI Column creation**

In [223]:
# creating a column for BMI
df['BMI'] = ((df['Weight']/(df['Height']**2))*10000) # multiplication by 10000 to account for meter to cm conversion
df

**Data visualization**

In [224]:
import seaborn as sns

# introductory question 2
# boxplot to represent silver medallist age distribution
sns.boxplot(x='Medal', y='Age', data = df)

In [225]:
# outlier percentage calculation
# inter-quartile range calculation
q1 = df['Age'].quantile(0.25)
q3 = df['Age'].quantile(0.75)
iqr = q3 - q1

# percentage of outliers for index column
p_out = (((df['Age'] < (q1 - 1.5 * iqr)) | (df['Age'] > (q3 + 1.5 * iqr))).sum() / len(df)) * 100
print('Percentage of outliers in age column : ', p_out, ' %')

In [226]:
import matplotlib.pyplot as plt

# introductory question 4
# creating a scatterplot to compare height & weight
plt.scatter(df['Height'], df['Weight'])
plt.ylabel('weight')
plt.xlabel('height')
plt.title('weight v/s height correlation')
plt.show()
# we can see a positive correlation between the two

In [227]:
# introductory question 5
# manual evaluation of the dataset shows that there exists more than 5 years of data for the three mentioned sports
# plotting the frequency data for (sex, sport) value pairs
df['Sport'].groupby(df['Sex']).value_counts().plot(kind = 'bar', ylabel = 'frequency', title = 'frequency data for (sex, sport) value pairs')

In [228]:
# task question 1
# getting number of years of participation by all teams

dict = {}

for index,row in df.iterrows():
  if row['Team'] not in dict.keys():
    dict[row['Team']] = 1
  else:
    dict[row['Team']] += 1

sorted_values = sorted(dict.values(),reverse=True) # Sort the values
sorted_dict = {}

for i in sorted_values:
    for k in dict.keys():
        if dict[k] == i:
            sorted_dict[k] = dict[k]
            break

sorted_dict

# Germany has highest number of medals won

In [229]:
ger = []
for index,row in df.iterrows():
  if row['Team'] == "Germany":
    ger.append(row)
ger_df = pd.DataFrame(ger)
ger_df
df_plot = ger_df.groupby(['Medal', 'Year']).size().reset_index().pivot(columns='Medal', index='Year', values=0)
df_plot.plot(kind='bar', stacked=True)

**Task question - New dataset creation**

In [230]:
df1 = list(df['Name'].value_counts())
df2 = list(df['Year'].groupby(df['Name']).count()) # dataframe with athlete and number of medals
list1 = list(zip(df1, df2))
result = []
for i,j in list1 :
    result.append(i/j)
df1 = pd.DataFrame(df['Name'].value_counts())
df1['Success'] = result
df1.to_csv('./task2_result.csv')

# therefore, Ole Einar Bjrndalen is the most successful athlete in the dataset